In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-11 13:29:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-06-11 13:29:34 (5.95 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [8]:
from pyspark import SparkFiles
# Load in employee.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

## Drop duplicates and incomplete rows

In [9]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

9002021
9001052
9001052


## Examine the schema

In [10]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Rename columns

In [11]:
# df1 = df.withColumnRenamed("Employee ID", "employee_id") \
#         .withColumnRenamed("Email", "email") \
#         .withColumnRenamed("Gender", "gender") \
#         .withColumnRenamed("Hire Date", "hire_date") \
#         .withColumnRenamed("DOB", "dob") \
#         .withColumnRenamed("Encrypted Password", "password")
# df1.show(5)
# NOTE: The above code from Homework Lesson 22.3.4 was not needed since all of the columns aligned to what was needed:)
df1=df
df1

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

## Create a new DataFrame for products 

In [12]:
products = df1.select(["product_id", "product_title"])
products.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B002OL2MU4|Garmin Portable f...|
|B00LAHPLTW|iPhone 6 case, Ca...|
|B003T91K94|BW 3.6mm Wide Ang...|
|B00ENI0JM4|ePartSolution-OEM...|
|B00EF1OGOG|Anker 2nd Generat...|
+----------+--------------------+
only showing top 5 rows



## Write DataFrame to RDS

In [13]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://rdsdb.cgx5vst360bw.us-east-2.rds.amazonaws.com:5432/dbforbigdata"
print("Need to change password below as per BZ Gmail email on 6/8/22 at 10:50pm")
print("change it from xxxxxxxx to temporary one 1 per BZ Gmail email on 6/8/22 at 10:50pm")
config = {"user":"postgres",
          "password": "xxxxxxxx",
          "driver":"org.postgresql.Driver"}

Need to change password below as per BZ Gmail email on 6/8/22 at 10:50pm
change it from xxxxxxxx to temporary one 1 per BZ Gmail email on 6/8/22 at 10:50pm


In [14]:
# Write DataFrame to table

products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

## Create a new DataFrame for review_id_table

In [15]:
review_id_table = df1.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10048FCFWVJOH|   34974850|B002OL2MU4|     471222439| 2011-04-07|
|R1008YTRJEH6U7|   30903818|B00LAHPLTW|     355012266| 2014-12-15|
|R100F3WWLNCX5S|   51491202|B003T91K94|     816619821| 2014-07-31|
|R100IJSLJ25VPG|    2572903|B00ENI0JM4|     653653620| 2014-11-30|
|R100JPDEL3NLYL|   37174514|B00EF1OGOG|     558284639| 2014-07-01|
|R100KWGZ8UZDA0|   13654097|B0079A29L2|     198567336| 2013-04-21|
|R100LHWI1PCRDO|   23612091|B007AQF0WK|      19433141| 2014-12-21|
|R100MJFI7QHR78|   44895028|B00PNUBGGY|     740916364| 2015-04-02|
|R100MKHZ325MI7|   35797687|B00DEKJEAA|      24966059| 2014-06-11|
|R100U625Y5RVRJ|   25608655|B00CI4VKB4|     689535231| 2013-11-21|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



## Write DataFrame to RDS

In [16]:
# Write DataFrame to review_id_table
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

## Create a new DataFrame for customers:

In [17]:
# Since customer_count isn't available in the Amazon supplied Datasets, first need to count customer occurences of customer_id, then rename columns, then 
customers = df1.select(["customer_id"])
customers.show(10)

customers = customers.groupBy("customer_id").count()
customers.show(10)

customers.orderBy("customer_id").select(["customer_id", "count"])
customers.show(10)

customers = customers.withColumnRenamed("count","customer_count")
customers.show(10)

customers.printSchema()

+-----------+
|customer_id|
+-----------+
|   34974850|
|   30903818|
|   51491202|
|    2572903|
|   37174514|
|   13654097|
|   23612091|
|   44895028|
|   35797687|
|   25608655|
+-----------+
only showing top 10 rows

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   49070323|    3|
|    4861668|    3|
|   44920760|    1|
|   33413426|    1|
|   32971810|    1|
|   24086603|   10|
|   31680089|    4|
|   25845557|    2|
|   47638296|    7|
|   47485107|    1|
+-----------+-----+
only showing top 10 rows

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   49070323|    3|
|    4861668|    3|
|   44920760|    1|
|   33413426|    1|
|   32971810|    1|
|   24086603|   10|
|   31680089|    4|
|   25845557|    2|
|   47638296|    7|
|   47485107|    1|
+-----------+-----+
only showing top 10 rows

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49070323|             3|
|    4861668|             3|
|   44920760|        

## Write DataFrame to RDS

In [18]:
# Write DataFrame to table
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

## Create a new DataFrame for vine_table


In [19]:
vine_table = df1.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10048FCFWVJOH|          5|            0|          0|   N|
|R1008YTRJEH6U7|          1|            1|          2|   N|
|R100F3WWLNCX5S|          5|            0|          0|   N|
|R100IJSLJ25VPG|          5|            0|          0|   N|
|R100JPDEL3NLYL|          3|            0|          0|   N|
|R100KWGZ8UZDA0|          1|            0|          1|   N|
|R100LHWI1PCRDO|          4|            0|          0|   N|
|R100MJFI7QHR78|          5|            0|          0|   N|
|R100MKHZ325MI7|          4|            2|          2|   N|
|R100U625Y5RVRJ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [20]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Change 'star_rating', 'helpful_votes', and 'total_votes' 
## in vine_table to Integer type

In [21]:
#from traitlets.traitlets import Integer
from pyspark.sql.types import IntegerType
## Convert star_rating from string to interger
vine_table = vine_table.withColumn("star_rating",vine_table["star_rating"].cast(IntegerType()))\
  .withColumn("helpful_votes",vine_table["helpful_votes"].cast(IntegerType()))\
  .withColumn("total_votes",vine_table["total_votes"].cast(IntegerType()))
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



## Write DataFrame to RDS

In [22]:
# Write DataFrame to table
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

## Printing out the counts for each table 
### (Will use it to compare the counts 
## via SQL querries)


In [25]:
print(products.count())
print(review_id_table.count())
print(customers.count())
print(vine_table.count())

9001052
9001052
5197356
9001052
